In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv('train.csv')

In [3]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
df.shape

(891, 12)

In [5]:
df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

## Data Preprocessing

In [6]:
name = df.pop('Name')

In [7]:
train = df.copy()

In [8]:
from sklearn.preprocessing import LabelEncoder

In [9]:
lb = LabelEncoder()

In [10]:
categorical_columns = [column for column in df.columns
                      if df[column].dtype == 'object']

for column in categorical_columns:
    train[column] = lb.fit_transform(train[column])

In [11]:
from sklearn.ensemble import BaggingRegressor
from sklearn.tree import DecisionTreeRegressor

In [12]:
def tree_imputation(df):
    missing_columns = [column for column in df.columns
                      if df[column].isnull().sum() > 0]
    
    non_missing_columns = [column for column in df.columns
                      if df[column].isnull().sum() == 0]
    
    for column in missing_columns:
        model = BaggingRegressor(DecisionTreeRegressor(), n_estimators=40,
                                max_samples=1.0, max_features=1.0, bootstrap=False,
                                 n_jobs=-1)
        
        column_missing = df[df[column].isnull()]
        temp = df.drop(df[df[column].isnull()].index, axis=0)
        
        X = temp.loc[:, non_missing_columns]
        y = temp[column]
        
        model.fit(X, y)
        
        y_pred = model.predict(column_missing[non_missing_columns])
        
        df.loc[column_missing.index, column] = y_pred
    
    return df

In [13]:
train_new = tree_imputation(train)

In [14]:
train_new.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,1,22.0,1,0,523,7.2500,147,2
1,2,1,1,0,38.0,1,0,596,71.2833,81,0
2,3,1,3,0,26.0,0,0,669,7.9250,147,2
3,4,1,1,0,35.0,1,0,49,53.1000,55,2
4,5,0,3,1,35.0,0,0,472,8.0500,147,2


In [15]:
train_new.isnull().sum()

PassengerId    0
Survived       0
Pclass         0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Cabin          0
Embarked       0
dtype: int64

In [16]:
train_new.drop('PassengerId', axis=1, inplace=True)

## XGBoost Model

In [17]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

In [18]:
X = train_new.drop('Survived', axis=1)
y = train_new['Survived']

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [20]:
# Initialize XGB
xgb = XGBClassifier(n_estimators=200, max_depth=4, random_state=1)

# Fit XGB to traning set
xgb.fit(X_train, y_train)

# Predict labels of test set, y_pred
y_pred = xgb.predict(X_test)

score = accuracy_score(y_pred, y_test)

print('Score: ' + str(score*100))

Score: 80.71748878923766


In [21]:
confusion_matrix(y_test, y_pred)

array([[115,  13],
       [ 30,  65]], dtype=int64)

Note: Prediction is keep changing because missing values prediction is changing as well.